In [7]:
import meshplot as mp
from coincidence_matching import get_export_id_types
import json
from zipfile import ZipFile
import pandas as pd
from tqdm import tqdm
import numpy as np
from automate import Part

In [ ]:
missing_data = []
num_mismatched_types = []

matching_types = np.zeros((4,4))

classmap = {
    'PK_CLASS_face':0,
    'PK_CLASS_edge':1,
    'PK_CLASS_vertex':2,
    'PK_CLASS_body':3
}

with ZipFile('../../../brepmatching/Geo.zip','r') as zf:
    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)
    
    data = variations
    data = data[(data.fail == 0) & (data.baselineNew != ' ') ]
    
    all_names = set(zf.namelist())
    
    for i in tqdm(range(len(data))):
        d = data.iloc[i]
        
        original_path = 'data/BrepsWithReference/' + d.ps_orig
        variation_path = 'data/BrepsWithReference/' + d.ps_var
        
        baseline_original_path = 'data/baseline/' + d.baselineOrig
        baseline_var_path = 'data/baseline/' + d.baselineNew
        
        path_gt_match = 'data/Matches/' + d.matchFile
        path_bl_match = 'data/baseline/' + d.baselineMatch
        
        data_exists = (original_path in all_names) and \
            (variation_path in all_names) and \
            (baseline_original_path in all_names) and \
            (baseline_var_path in all_names) and \
            (path_gt_match in all_names) and \
            (path_bl_match in all_names)
        
        # Verify that all data files are in the archive
        if not data_exists:
            missing_data.append(i)
            continue
        
        # Load parts and match files
        with zf.open(original_path,'r') as f:
            original_part_data = f.read().decode('utf-8')
        with zf.open(variation_path,'r') as f:
            variation_part_data = f.read().decode('utf-8')
        with zf.open(path_gt_match, 'r') as f:
            gt_match = json.load(f)
            
        orig_types = get_export_id_types(original_part_data)
        var_types = get_export_id_types(variation_part_data)
                
        for match in gt_match.values():
            orig_id = match['val1']
            var_id = match['val2']
            
            o_idx = classmap[orig_types[orig_id]]
            v_idx = classmap[var_types[var_id]]
            
            matching_types[o_idx,v_idx] = matching_types[o_idx,v_idx] + 1
            
            

 97%|█████████████████████████████████████████████████████████████████████████████████████████▍  | 6604/6797 [01:26<00:03, 61.92it/s]

In [9]:
(np.array(num_mismatched_types) > 0).sum()

5836

In [13]:
mismatches.astype(int)

array([[    0, 59041,   113,  2338],
       [65353,     0, 32918,   760],
       [  881, 36207,     0,    55],
       [ 2268,   343,     5,     0]])